# Basic Text Classification w/ Keras

https://www.tensorflow.org/tutorials/keras/text_classification

In [9]:
from pathlib import Path
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers, losses, preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Binary Classification (Dataset: IMDB Reviews)

Fetch and Download Dataset

In [10]:
dataset_url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

In [11]:
datasets_dir = Path(os.path.abspath('')).parent.joinpath('datasets')
datasets_dir.mkdir(parents=True, exist_ok=True)

In [12]:
dataset = tf.keras.utils.get_file("aclImdb_v1", dataset_url,
                                  untar=True, 
                                  cache_dir=datasets_dir,
                                  cache_subdir='')

In [13]:
imdb_dataset_dir = datasets_dir.joinpath('aclimdb')

In [14]:
imdb_train_dataset = imdb_dataset_dir.joinpath('train')
imdb_test_dataset = imdb_dataset_dir.joinpath('test')

In [15]:
os.listdir(imdb_train_dataset)

['urls_unsup.txt',
 'neg',
 'urls_pos.txt',
 'unsup',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [16]:
"""Positive IMDB Reviews Dataset"""
imdb_train_dataset_pos = imdb_train_dataset.joinpath('pos')

In [17]:
"""Negative IMDB Reviews Dataset"""
imdb_train_dataset_neg = imdb_train_dataset.joinpath('neg')

In [18]:
"""Drops the 'unsup' dataset"""
imdb_train_dataset_unsup = imdb_train_dataset.joinpath('unsup')
shutil.rmtree(imdb_train_dataset_unsup, ignore_errors=True)

## Loading the Dataset

In [19]:
"""
Loads the Dataset in Batches/Lists of 32 entries each,
meaning that each iteration over the dataset will return 32 reviews
"""
batch_size = 32
seed = 42

### Loading Training Dataset

In [20]:
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    imdb_train_dataset,
    batch_size=batch_size,
    seed=seed,
    validation_split=0.2,
    subset='training'
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


### Data Analysis Training Dataset

In [21]:
"""Available Classes"""
raw_train_ds.class_names

['neg', 'pos']

In [22]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        tokenized_label = label_batch.numpy()[i]
        corpus=text_batch.numpy()[i]
        class_name = raw_train_ds.class_names[tokenized_label]
        print(f"Label: {tokenized_label} ({class_name})")
        print(f"Review: {corpus}")
        print()

Label: 0 (neg)
Review: b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'

Label: 0 (neg)
Review: b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life 

### Loading Validation Dataset

In [23]:
raw_validation_ds = tf.keras.preprocessing.text_dataset_from_directory(
    imdb_train_dataset,
    batch_size=batch_size,
    seed=seed,
    validation_split=0.2,
    subset='validation'
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


#### Loading Test Dataset

In [24]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    imdb_test_dataset,
    batch_size=batch_size
)

Found 25000 files belonging to 2 classes.


## Dataset Preprocessing

### Standarization Function

In [25]:
def standardize_fn(input_data):
    lowercase_corpus = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(input=lowercase_corpus, 
                                             pattern='<br />', 
                                             rewrite='')

    return tf.strings.regex_replace(input=stripped_html, 
                                    pattern='[%s]' % re.escape(string.punctuation), 
                                    rewrite='')

### Tokenization & Vectorization

In [26]:
"""Define the size of the vocabulary (the max number of words to be tokenized)"""
max_features = 100000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=standardize_fn,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

### Testing & Visualization

In [27]:
train_text = raw_train_ds.map(lambda text, class_label: text)

In [28]:
vectorize_layer.adapt(train_text)

In [29]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
review, label = text_batch[0], label_batch[0]

In [30]:
print("Review: ", review)

Review:  tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br /

In [31]:
print("Standarized Review: ", standardize_fn(review))

Standarized Review:  tf.Tensor(b'silent night deadly night 5 is the very last of the series and like part 4 its unrelated to the first three except by title and the fact that its a christmasthemed horror flickexcept to the oblivious theres some obvious things going on heremickey rooney plays a toymaker named joe petto and his creepy sons name is pino ring a bell anyone now a little boy named derek heard a knock at the door one evening and opened it to find a present on the doorstep for him even though it said dont open till christmas he begins to open it anyway but is stopped by his dad who scolds him and sends him to bed and opens the gift himself inside is a little red ball that sprouts santa arms and a head and proceeds to kill dad oops maybe he should have left wellenough alone of course derek is then traumatized by the incident since he watched it from the stairs but he doesnt grow up to be some killer santa he just stops talkingtheres a mysterious stranger lurking around who seem

In [32]:
print("Label: ", raw_train_ds.class_names[label])

Label:  neg


In [33]:
print("Vectorized review: ", vectorize_text(review, label))

Vectorized review:  (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[ 1270,   312,  2355,   312,   682,     7,     2,    51,   227,
            5,     2,   203,     3,    37,   170,   725,    29,  5496,
            6,     2,    86,   295,   541,    32,   427,     3,     2,
          185,    12,    29,     4,     1,   191,     1,     6,     2,
         8333,   221,    46,   573,   176,   167,    20, 94681,  5434,
          284,     4, 25205,   752,   965, 25879,     3,    24,   931,
         2275,   396,     7, 14156,  1655,     4,  3734,   252,   157,
            4,   110,   437,   752,  3561,   538,     4,  3648,    30,
            2,  1347,    28,  2121,     3,  2877,     9,     6,   160,
            4,   998,    20,     2, 16825,    15,    88,    54,   149,
            9,   290,    89,   960,  2273,   983,    27,   766,     6,
          960,     9,   687,    18,     7,  2138,    32,    24,  1265,
           36, 33282,    88,     3,  3249,    88,     6,  1411,     3,
        

In [37]:
print("{token} ==> {word}".format(token=11, word=vectorize_layer.get_vocabulary()[11]))
print("{token} ==> {word}".format(token=57, word=vectorize_layer.get_vocabulary()[57]))
print("{token} ==> {word}".format(token=1270, word=vectorize_layer.get_vocabulary()[1270]))

print("Vocabulary Size: {size}".format(size=len(vectorize_layer.get_vocabulary())))

11 ==> i
57 ==> would
1270 ==> silent
Vocabulary Size: 100000


### Applying Vectorization on Training/Validation/Test Datasets

In [43]:
train_ds = raw_train_ds.map(vectorize_text)
validation_ds = raw_validation_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

### Caching Datasets to RAM

In [46]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds.cache().prefetch(buffer_size=AUTOTUNE)

<PrefetchDataset shapes: ((None, 250), (None,)), types: (tf.int64, tf.int32)>

## Create the Model